In [1]:
# optional installs if these packages are not included in your configuration
%pip install ipython jupyter
%pip install dask[complete]
%pip install dask
%pip install dask[dataframe]
%pip install pandas




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.4/257.4 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 30.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.8/162.8 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.4/383.4 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 20.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━

In [8]:
import dask.dataframe as dd
import os
from IPython.display import display
import json
import pandas as pd
import numpy as np

In [9]:
# directory paths to the folders that hold the 2016 and 2017 flight data
flight_data_dir_2016 = r"C:\Users\raaja\OneDrive\Documents\maichu development\ml project #1\2016\2016_DATA"
flight_data_dir_2017 = r"C:\Users\raaja\OneDrive\Documents\maichu development\ml project #1\2017\2017_DATA"

In [ ]:
# a function to extract information given a data directory path as an argument
def load_flight_data(data_dir):
    # initialize an empty Dask dataframe
    combined_df = None

    # iterate throughe each file in the given directory path
    for file in os.listdir(data_dir):
        # read into the file if it is a csv file
        if file.endswith('.csv'):
            # create the full filepath for that file given the filename
            file_path = os.path.join(data_dir, file)

            # based off of debugging messages, set dtype as python interpreter cannot predict the types of the columns if they are mixed
            df = dd.read_csv(file_path, dtype={'ArrTime': 'float64',
       'ArrivalDelayGroups': 'float64',
       'CancellationCode': 'object',
       'DepTime': 'float64',
       'DepartureDelayGroups': 'float64',
       'Div1Airport': 'object',
       'Div1TailNum': 'object',
       'Div2Airport': 'object',
       'Div2TailNum': 'object',
       'Div3Airport': 'object',
       'WheelsOff': 'float64',
       'WheelsOn': 'float64',
       'DivAirportLandings': 'float64'})
            
            # actually set the data frame as a Dask data frame here appropriately
            if combined_df is None:
                combined_df = df
            else:
                combined_df = dd.concat([combined_df, df])

    return combined_df

In [ ]:
# call the above function on 2016 and 2017 directory paths and assign them to a data frame each
flight_data_2016 = load_flight_data(flight_data_dir_2016)
flight_data_2017 = load_flight_data(flight_data_dir_2017)

In [ ]:
# merge both years data frames into a single Dask data frame
flight_data = dd.concat([flight_data_2016, flight_data_2017], ignore_index=True)

In [ ]:
# make a list of the desired columns in the data frame
desired_columns = ["FlightDate", "Quarter", "Year", "Month", "DayofMonth",
                  "DepTime", "DepDel15", "CRSDepTime", "DepDelayMinutes", "Origin", "Dest",
                  "OriginAirportID", "DestAirportID", "ArrTime", "CRSArrTime",
                  "ArrDel15", "ArrDelayMinutes"]


In [ ]:
# make a new data frame with only the filtered columns by dropping the unrequired columns
filtered_flight_data = flight_data.drop(columns=list(set(flight_data.columns) - set(desired_columns)))

In [ ]:
# compute the Dask data frame into a Pandas data frame (note: computationally expensive)
dataf = filtered_flight_data.compute()

,Year,Quarter,Month,DayofMonth,FlightDate,OriginAirportID,Origin,DestAirportID,Dest,CRSDepTime,DepTime,DepDelayMinutes,DepDel15,CRSArrTime,ArrTime,ArrDelayMinutes,ArrDel15
0,2016,1,1,6,2016-01-06,11298,DFW,11433,DTW,1100,1057.0,0.0,0.0,1438,1432.0,0.0,0.0
1,2016,1,1,7,2016-01-07,11298,DFW,11433,DTW,1100,1056.0,0.0,0.0,1438,1426.0,0.0,0.0
2,2016,1,1,8,2016-01-08,11298,DFW,11433,DTW,1100,1055.0,0.0,0.0,1438,1445.0,7.0,0.0
3,2016,1,1,9,2016-01-09,11298,DFW,11433,DTW,1100,1102.0,2.0,0.0,1438,1433.0,0.0,0.0
4,2016,1,1,10,2016-01-10,11298,DFW,11433,DTW,1100,1240.0,100.0,1.0,1438,1631.0,113.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153329,2017,3,9,25,2017-09-25,13930,ORD,10721,BOS,1018,1332.0,194.0,1.0,1335,1647.0,192.0,1.0
153330,2017,3,9,25,2017-09-25,12478,JFK,12954,LGB,1828,1822.0,0.0,0.0,2135,2135.0,0.0,0.0
153331,2017,3,9,25,2017-09-25,12451,JAX,11697,FLL,630,623.0,0.0,0.0,743,732.0,0.0,0.0
153332,2017,3,9,25,2017-09-25,11697,FLL,12451,JAX,700,654.0,0.0,0.0,812,802.0,0.0,0.0


In [ ]:
# a list of all the airports we are interested in
desired_airports = ["ATL", "CLT", "DEN", "DFW", "EWR",
                    "IAH", "JFK", "LAS", "LAX", "MCO",
                    "MIA", "ORD", "PHX", "SEA", "SFO"]

# a list of all the years within each airport directory we are interested in holding the data of
desired_years = [2016, 2017]

In [ ]:
# a directory path to a folder with all the airport folders 
airports_dir_path = r"C:\Users\raaja\OneDrive\Documents\maichu development\ml project #1\weather"


In [ ]:
# initialize an empty data frame
weather_data_df = pd.DataFrame()

In [ ]:
# os.listdir returns a list of every single file and airport in a filepath, which this for loop would iterate through
for airport in os.listdir(airports_dir_path):
    # if the filename returned by os.listdir is an airport that is in the desired_columns list...
    if airport in desired_airports:
        # generate a full file path to that folder to iterate through
        full_airport_path = os.path.join(airports_dir_path, airport)

        # for each file in eaach airport folder ...
        for filename in os.listdir(full_airport_path):

            # if that file contains one of the years that we are interested in for the model...
            if any(str(year) in filename for year in desired_years):
                # generate a full filepath to that file to read into
                file_path = os.path.join(full_airport_path, filename)

                # create airport column based on folder name
                airport_name = airport

                # syntax to read into a json file given a file_path
                with open(file_path, 'r') as f:
                    data = json.load(f)

                    # since weather is given as a list, we can iterate through it to find the columns we want
                    for daily_weather in data['data']['weather']:

                        # extract date from "weather" column
                        date = daily_weather['date']

                        # create a data structure to hold the  columns from "hourly" data
                        hourly_data = daily_weather['hourly']

                        # create a DataFrame from hourly data
                        hourly_df = pd.DataFrame(hourly_data)

                        # select the desired columns
                        hourly_df = hourly_df[["windspeedKmph", "winddirDegree", "weatherCode", "precipMM",
                                               "visibility", "pressure", "cloudcover", "DewPointF",
                                               "WindGustKmph", "tempF", "WindChillF", "humidity", "time"]]

                        # add the airport and date columns
                        hourly_df["airport"] = airport_name
                        hourly_df["date"] = date

                        # Concatenate with existing data
                        weather_data_df = pd.concat([weather_data_df, hourly_df], ignore_index=True)


,windspeedKmph,winddirDegree,weatherCode,precipMM,visibility,pressure,cloudcover,DewPointF,WindGustKmph,tempF,WindChillF,humidity,time,airport,date
0,11,318,176,0.1,10,1023,100,47,17,49,46,91,0,ATL,2016-01-01
1,13,317,176,0.0,10,1023,100,44,22,46,42,92,100,ATL,2016-01-01
2,14,315,122,0.0,10,1023,100,41,26,43,38,92,200,ATL,2016-01-01
3,16,314,122,0.0,10,1023,100,38,30,40,33,93,300,ATL,2016-01-01
4,17,314,122,0.0,10,1023,100,38,30,40,33,93,400,ATL,2016-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263155,26,296,113,0.0,10,1014,0,57,27,67,67,70,1900,SFO,2017-09-30
263156,26,298,113,0.0,10,1014,0,58,27,67,67,71,2000,SFO,2017-09-30
263157,26,300,113,0.0,10,1014,0,58,27,67,67,73,2100,SFO,2017-09-30
263158,23,302,113,0.0,10,1014,0,58,25,66,66,74,2200,SFO,2017-09-30


In [ ]:
# a function that rounds a military time to the closest hour
def roundoff(time):
    hour = time // 100
    minute = time % 100
    if (minute > 30):
        hour += 1

    hour = (hour % 24) * 100
    return hour

# print(roundoff(2259))

In [ ]:
# drop all the nul values in the flight data frame
dataf.dropna(inplace=True)


In [ ]:
# round the CRS departure time to the closest hour to correspond with the most accurate weather data
dataf["roundDepTime"] = dataf["CRSDepTime"].apply(roundoff)
# dataf["roundArrTime"] = dataf["CRSArrTime"].apply(roundoff)

In [ ]:
# drop all the rows where the origin airport and the destination airport is not an airport in desired airports
dataf = dataf[dataf["Origin"].isin(desired_airports) & dataf["Dest"].isin(desired_airports)]

# if you are making a training model only based off of the origin airports weather then there is no need to drop any destination airports that we do not have weather data on
# dataf= dataf[dataf["Origin"].isin(desired_airports)]
# dataf = dataf.drop(columns=list(set(["roundArrTime"])))



In [ ]:
# define columns to typecast
int_cols = weather_data_df.columns.drop(['airport', 'date'])

# try converting to integer, handle errors with 'coerce'
try:
  weather_data_df[int_cols] = weather_data_df[int_cols].apply(pd.to_numeric, errors='coerce')
  print("Successfully converted convertible columns to int64!")
  print(weather_data_df.dtypes)
except:
  print("Error: Some columns might not be convertible to int64.")

# check data types after conversion
# print(weather_data_df.dtypes)


Successfully converted convertible columns to int64!
windspeedKmph      int64
winddirDegree      int64
weatherCode        int64
precipMM         float64
visibility         int64
pressure           int64
cloudcover         int64
DewPointF          int64
WindGustKmph       int64
tempF              int64
WindChillF         int64
humidity           int64
time               int64
airport           object
date              object
dtype: object


In [ ]:
# merge the flight and the weather data set based on the date, time, and airport
merged_df = pd.merge(dataf, weather_data_df, how='left', left_on=['FlightDate', 'roundDepTime', 'Origin'], right_on=['date', 'time', 'airport'])


In [ ]:
# save the merged dataframe to a csv file
merged_df.to_csv(r'merged_data.csv', index=False)

In [ ]:
# Assuming your DataFrame is named 'merged_df'
merged_df['IsDelayed'] = merged_df['ArrDelayMinutes'].gt(0).map({True: "Delayed", False: "Not Delayed"})


,Year,Quarter,Month,DayofMonth,FlightDate,OriginAirportID,Origin,DestAirportID,Dest,CRSDepTime,...,cloudcover,DewPointF,WindGustKmph,tempF,WindChillF,humidity,time,airport,date,IsDelayed
0,2016,1,1,1,2016-01-01,14747,SEA,12478,JFK,745,...,0,23,8,34,29,65,800,SEA,2016-01-01,Delayed
1,2016,1,1,2,2016-01-02,14747,SEA,12478,JFK,745,...,0,25,8,37,33,63,800,SEA,2016-01-02,Delayed
2,2016,1,1,3,2016-01-03,14747,SEA,12478,JFK,745,...,0,20,7,32,28,64,800,SEA,2016-01-03,Not Delayed
3,2016,1,1,4,2016-01-04,14747,SEA,12478,JFK,745,...,93,33,8,35,31,94,800,SEA,2016-01-04,Not Delayed
4,2016,1,1,5,2016-01-05,14747,SEA,12478,JFK,710,...,100,35,10,38,35,91,700,SEA,2016-01-05,Not Delayed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1851431,2017,3,9,25,2017-09-25,14771,SFO,12478,JFK,2045,...,0,63,20,72,72,72,2100,SFO,2017-09-25,Not Delayed
1851432,2017,3,9,25,2017-09-25,11618,EWR,13204,MCO,1205,...,8,68,8,83,83,60,1200,EWR,2017-09-25,Not Delayed
1851433,2017,3,9,25,2017-09-25,13204,MCO,11618,EWR,1141,...,17,74,10,84,84,71,1200,MCO,2017-09-25,Not Delayed
1851434,2017,3,9,25,2017-09-25,12889,LAS,12478,JFK,2159,...,1,27,19,64,64,24,2200,LAS,2017-09-25,Not Delayed


In [ ]:
# save the merged dataframe to a csv file
merged_df.to_csv(r'merged_data2.csv', index=False)